# Feat 1 Check  
Here we go over quality checking our feat1 directories made by FSL!  
   
Reference: [Checking your level 1 - mumfordbrainstats](https://www.youtube.com/watch?v=kW1xgGs4NUU&list=PLB2iAtgpI4YHlH4sno3i3CUjCofI38a-3&index=13)  
Checklist:  
* Create html  
* Check for exsistence of expected files  
    * zstats   
    * look into log files (errors, etc...)  



When looking at html: 
* Regressors look like you want  
* check pairwise colinearity

In [37]:
import os, glob
import subprocess

In [40]:
# get feat1 directories
bids_path=input("Enter target study bids path: ")
#/projects/niblab/bids_projects/Experiments/bbx/bids

Enter target study bids path: /projects/niblab/bids_projects/Experiments/bbx/bids


In [41]:
subjects = sorted(glob.glob(os.path.join(bids_path, "derivatives/sub-*/ses-1")))

In [28]:
outfile = os.path.join(bids_path, "derivatives/testing_feat1_qc.html")
f = open(outfile, "w")

for sub_path in subjects[:16]:
    sub_id = sub_path.split("/")[-2]
    f.write("<h2>Subject: %s\n</h2>"%sub_id)
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    for f1_dir in feat1_dirs:
        run_id = f1_dir.split("/")[-1].split("_")[2]
        f.write("<h4>Directory: %s\n</h4>"%f1_dir)

        design_img1 = "%s/design.png"%f1_dir
        if os.path.exists(design_img1):
            f.write("<p><img src=%s alt=img1"%(design_img1))
            f.write("\n")
        design_img2 = "%s/design_cov.png"%f1_dir
        if os.path.exists(design_img2):
            f.write("<p><IMG SRC=%s alt=img2"%(design_img2))
            f.write("\n")
        func2high_img = "%s/reg/example_func2highres.png"%f1_dir
        if os.path.exists(func2high_img):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img3>"%(func2high_img))
            f.write("\n")
        func2std_img = "%s/reg/example_func2standard.png"%f1_dir
        if os.path.exists(func2std_img ):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img4>"%(func2std_img))
            f.write("\n")
        high2std_img = "%s/reg/highres2standard.png"%f1_dir
        if os.path.exists(high2std_img ):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img5>"%(high2std_img))
            f.write("\n")
        
        
        
f.close()

### Checking for zstats

In [42]:
for sub_path in subjects[:16]:
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    for run_path in feat1_dirs:
        zstats=glob.glob(os.path.join(run_path, 'stats/zstat*nii.gz'))
        if not zstats:
            print("Missing zstats for subject: \t", sub_path.split("/")[-2] )
            

Missing zstats for subject: 	 sub-010


### Checking for errors in logs

In [60]:
for sub_path in subjects:
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    for run_dir in feat1_dirs:
        html = os.path.join(run_dir, "report_log.html")
        cmd = subprocess.Popen(["grep", "-i", "'error'", "%s"%html],  stdout=subprocess.PIPE, shell=True)
        output, err = cmd.communicate()
        if not output:
            pass
        else:
            print(output)